In [1]:
import numpy as np
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
import time
from src.environments import ConPricingGame
import src.globals as gl
import src.classes as cl

2023-11-06 17:41:41.224529: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-06 17:41:41.925469: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-06 17:41:42.576454: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-06 17:41:42.576496: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-06 17:41:42.576514: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
num_procs = 15

global job_name
job_name = "lr"
gl.initialize()
cl.set_job_name(job_name)

alg = SAC
env_class = ConPricingGame

lrs = np.linspace(0.00001, 0.001, 20)


costs=[gl.LOW_COST,gl.HIGH_COST]

adv_mixed_strategy = cl.MixedStrategy(strategies_lst=[cl.Strategy(cl.StrategyType.static, cl.myopic, name="myopic"), cl.Strategy(
    cl.StrategyType.static, cl.myopic, name="guess", first_price=132)], probablities_lst=[0.5, 0.5])


In [ ]:
for lr in lrs:
    for iter in range(3):
        cl.prt(f"\nlr = {lr:.5f}, iter= {iter+1} started. {time.ctime(time.time())} ---------\n")
        alg_name = f"{job_name}_{lr:5f}"
        

        seed = int(time.time())

        model_name = f"{alg_name}-{str(seed)}"
        models_dir = f"{gl.MODELS_DIR}/{model_name}"
        log_dir = f"{gl.LOG_DIR}/{model_name}"

        number_episodes = 4_000_000
        train_env = make_vec_env(env_class, n_envs=num_procs, seed=seed, vec_env_cls=SubprocVecEnv, env_kwargs=dict(
            tuple_costs=costs, adversary_mixed_strategy=adv_mixed_strategy))
        model = alg('MlpPolicy', train_env, learning_rate=lr,
                    verbose=0, tensorboard_log=log_dir, gamma=gl.GAMMA, target_entropy=0)

        start = time.time()
        # for i in range(gl.NUM_MODEL_SAVE):
        # tmp = (number_episodes/gl.NUM_MODEL_SAVE)
        # model.learn(total_timesteps=tmp, reset_num_timesteps=False,
        #             tb_log_name=model_name)
        # model.save(os.path.join(models_dir, str(tmp*(i+1))))
        model.learn(total_timesteps=number_episodes, tb_log_name=model_name)
        model.save(models_dir)
        cl.prt(f"model {models_dir} is saved!\n")

        running_time = time.time() - start

    
        pricing_game = env_class(
            tuple_costs=costs, adversary_mixed_strategy=adv_mixed_strategy)
        model_strategy = cl.Strategy(strategy_type=cl.StrategyType.sb3_model,
                                    model_or_func=alg, name=model_name, action_step=pricing_game.action_step)
        mean_payoffs=[0]*len(adv_mixed_strategy.strategies)
        total_mean=0
        for strategy_index in range(len(adv_mixed_strategy.strategies)):
            if adv_mixed_strategy.strategy_probs[strategy_index] > 0:
                pricing_game = env_class(tuple_costs=costs, adversary_mixed_strategy=(adv_mixed_strategy.strategy_probs[strategy_index]).to_mixed_strategy())
                payoffs = []
                for i in range(gl.NUM_STOCHASTIC_ITER):
                    # returns = algorithm.play_trained_agent(adversary=(
                    #     (adv_mixed_strategy._strategies[strategy_index]).to_mixed_strategy()), iterNum=gl.num_stochastic_iter)
                    payoffs.append(model_strategy.play_against(
                        env=pricing_game, adversary=adv_mixed_strategy.strategies[strategy_index]))
                    data = [model_name, number_episodes, ("L" if (costs[0] < costs[1]) else "H"), pricing_game.adversary_strategy.name,
                            sum(pricing_game.profit[0]), sum(pricing_game.profit[1]),  str(
                                pricing_game.profit[0]), str(pricing_game.actions), str(pricing_game.prices[0]),
                            str(pricing_game.prices[1]), str(pricing_game.demand_potential[0]), str(
                                pricing_game.demand_potential[1]), lr, gl.NUM_ADV_HISTORY,
                            gl.TOTAL_STAGES, pricing_game.action_step, gl.NUM_ACTIONS, gl.GAMMA, False, seed, num_procs, running_time]
                    cl.write_results(data)

                mean_payoffs[strategy_index] = (np.array(payoffs).mean(axis=0))[0]
                total_mean+= mean_payoffs[strategy_index]*adv_mixed_strategy.strategy_probs[strategy_index]

                

        data = [model_name, number_episodes, ("L" if (costs[0] < costs[1]) else "H"), str(adv_mixed_strategy)+str(mean_payoffs), total_mean, 0,
                lr, gl.NUM_ADV_HISTORY, gl.TOTAL_STAGES, pricing_game.action_step, gl.NUM_ACTIONS, gl.GAMMA, seed, num_procs, running_time,
                time.ctime(time.time())]

        cl.write_agents(data)
        cl.prt(f"results of model {model_name} are written!\n")

2023-11-06 17:42:28.656548: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-06 17:42:28.656549: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-06 17:42:28.656552: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-06 17:42:28.656549: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly 